In [1]:
# ==== Install dependencies ====
!pip install gspread oauth2client moviepy gTTS pillow numpy

# ==== Imports ====
import os, io, textwrap
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from gtts import gTTS
from moviepy.editor import (
    ImageClip, AudioFileClip, concatenate_videoclips
)
from moviepy.video.fx.all import fadein, fadeout

# ==== Google Sheet Connect ====
import gspread
from oauth2client.service_account import ServiceAccountCredentials

SERVICE_JSON = "/credentials/service_account.json"   # upload your JSON file here
SHEET_URL = "https://docs.google.com/spreadsheets/d/1NeOa8Gv5rT6r5cuIdaSA2UMsX0E3fFJyZfJq6JJ2lhY/edit"

scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive",
]
creds = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_JSON, scope)
client = gspread.authorize(creds)
sheet = client.open_by_url(SHEET_URL).sheet1

rows = sheet.get_all_values()
start_index = 1 if rows and rows[0][0].strip().lower() == "title" else 0
title, description = rows[start_index][0], rows[start_index][1]
print("Title:", title)
print("Description:", description)

# ==== Config ====
VIDEO_W, VIDEO_H = 1280, 720
FPS = 30

# ==== Helper: Render text with Pillow ====
def text_image_clip(title, body, duration=25, bg_color=(30,30,50)):
    img = Image.new("RGB", (VIDEO_W, VIDEO_H), bg_color)
    draw = ImageDraw.Draw(img)

    try:
        title_font = ImageFont.truetype("DejaVuSans-Bold.ttf", 64)
        body_font = ImageFont.truetype("DejaVuSans.ttf", 38)
    except:
        title_font = ImageFont.load_default()
        body_font = ImageFont.load_default()

    # Title bounding box
    tb = draw.textbbox((0,0), title, font=title_font)
    tw, th = tb[2]-tb[0], tb[3]-tb[1]
    draw.text(((VIDEO_W-tw)//2, int(VIDEO_H*0.2)), title, font=title_font, fill="white")

    # Body text
    wrapped = textwrap.fill(body, width=55)
    bb = draw.multiline_textbbox((0,0), wrapped, font=body_font)
    bw, bh = bb[2]-bb[0], bb[3]-bb[1]
    draw.multiline_text(((VIDEO_W-bw)//2, int(VIDEO_H*0.5)), wrapped, font=body_font, fill="lightgrey", align="center")

    clip = ImageClip(np.array(img)).set_duration(duration)
    clip = fadein(clip, 0.5)
    clip = fadeout(clip, 0.5)
    return clip

# ==== Narration ====
def narrate(text, path):
    tts = gTTS(text=text, lang="en")
    tts.save(path)
    return path

# ==== Build Video ====
def build_video(title, description, out="output.mp4", max_duration=180):
    sentences = [s.strip() for s in description.split(".") if s.strip()]
    if len(sentences) < 7:
        while len(sentences) < 7:
            sentences.append(sentences[-1] if sentences else "Insight.")

    narration_text = " ".join(sentences)
    narr_path = narrate(narration_text, "/content/narration_full.mp3")
    narr_audio = AudioFileClip(narr_path)

    # ✅ Actual video duration = min(audio duration, max_duration)
    total_duration = min(narr_audio.duration, max_duration)
    per_seg = total_duration / 7

    clips = []
    current_t = 0.0
    titles = ["Hook","Intro","Details","Examples","Benefits","Challenges","Future"]
    for i in range(7):
        seg_dur = min(per_seg, narr_audio.duration - current_t)
        if seg_dur <= 0:
            break
        scene = text_image_clip(titles[i], sentences[i], seg_dur)
        a_sub = narr_audio.subclip(current_t, current_t+seg_dur)
        scene = scene.set_audio(a_sub)
        clips.append(scene)
        current_t += seg_dur

    final = concatenate_videoclips(clips, method="compose")
    final.write_videofile(out, fps=FPS, codec="libx264", audio_codec="aac", verbose=False, logger=None)
    return out

# ==== Run ====
mp4_path = build_video(title, description, out="/content/text2video_sheet.mp4", max_duration=180)
print("Video generated:", mp4_path)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.3.1
    Uninstalling click-8.3.1:
      Successfully uninstalled click-8.3.1


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



FileNotFoundError: [Errno 2] No such file or directory: '/credentials/service_account.json'

In [5]:

# ==== Imports ====
import os, io, textwrap
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from gtts import gTTS
from moviepy.editor import (
    ImageClip, AudioFileClip, concatenate_videoclips
)
from moviepy.video.fx.all import fadein, fadeout

# ==== Google Sheet Connect ====
import gspread
from oauth2client.service_account import ServiceAccountCredentials
SERVICE_JSON = "credentials/texto-474417-10f49ccb68ba.json"
# SERVICE_JSON = "/texto-474417-10f49ccb68ba.json"   # upload your JSON file here
SHEET_URL = "https://docs.google.com/spreadsheets/d/1NeOa8Gv5rT6r5cuIdaSA2UMsX0E3fFJyZfJq6JJ2lhY/edit"

scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive",
]
creds = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_JSON, scope)
client = gspread.authorize(creds)
sheet = client.open_by_url(SHEET_URL).sheet1

rows = sheet.get_all_values()
start_index = 1 if rows and rows[0][0].strip().lower() == "title" else 0
title, description = rows[start_index][0], rows[start_index][1]
print("Title:", title)
print("Description:", description)

# ==== Config ====
VIDEO_W, VIDEO_H = 1280, 720
FPS = 30

# ==== Helper: Render text with Pillow ====
def text_image_clip(title, body, duration=25, bg_color=(30,30,50)):
    img = Image.new("RGB", (VIDEO_W, VIDEO_H), bg_color)
    draw = ImageDraw.Draw(img)

    try:
        title_font = ImageFont.truetype("DejaVuSans-Bold.ttf", 64)
        body_font = ImageFont.truetype("DejaVuSans.ttf", 38)
    except:
        title_font = ImageFont.load_default()
        body_font = ImageFont.load_default()

    # Title bounding box
    tb = draw.textbbox((0,0), title, font=title_font)
    tw, th = tb[2]-tb[0], tb[3]-tb[1]
    draw.text(((VIDEO_W-tw)//2, int(VIDEO_H*0.2)), title, font=title_font, fill="white")

    # Body text
    wrapped = textwrap.fill(body, width=55)
    bb = draw.multiline_textbbox((0,0), wrapped, font=body_font)
    bw, bh = bb[2]-bb[0], bb[3]-bb[1]
    draw.multiline_text(((VIDEO_W-bw)//2, int(VIDEO_H*0.5)), wrapped, font=body_font, fill="lightgrey", align="center")

    clip = ImageClip(np.array(img)).set_duration(duration)
    clip = fadein(clip, 0.5)
    clip = fadeout(clip, 0.5)
    return clip

# ==== Narration ====
def narrate(text, path):
    tts = gTTS(text=text, lang="en")
    tts.save(path)
    return path

# ==== Build Video ====
def build_video(title, description, out="output.mp4", max_duration=180):
    sentences = [s.strip() for s in description.split(".") if s.strip()]
    if len(sentences) < 7:
        while len(sentences) < 7:
            sentences.append(sentences[-1] if sentences else "Insight.")

    narration_text = " ".join(sentences)
    narr_path = narrate(narration_text, "narration/narration_full.mp3")
    narr_audio = AudioFileClip(narr_path)

    # ✅ Actual video duration = min(audio duration, max_duration)
    total_duration = min(narr_audio.duration, max_duration)
    per_seg = total_duration / 7

    clips = []
    current_t = 0.0
    titles = ["Hook","Intro","Details","Examples","Benefits","Challenges","Future"]
    for i in range(7):
        seg_dur = min(per_seg, narr_audio.duration - current_t)
        if seg_dur <= 0:
            break
        scene = text_image_clip(titles[i], sentences[i], seg_dur)
        a_sub = narr_audio.subclip(current_t, current_t+seg_dur)
        scene = scene.set_audio(a_sub)
        clips.append(scene)
        current_t += seg_dur

    final = concatenate_videoclips(clips, method="compose")
    final.write_videofile(out, fps=FPS, codec="libx264", audio_codec="aac", verbose=False, logger=None)
    return out

# ==== Run ====
mp4_path = build_video(title, description, out="narration/text2video_sheet.mp4", max_duration=180)
print("Video generated:", mp4_path)


FileNotFoundError: [Errno 2] No such file or directory: 'credentials/texto-474417-10f49ccb68ba.json'